In [1]:
import requests
import pandas as pd
import time
import numpy as np
import ast
from datetime import ( timedelta, date)
import datetime

import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import (MarketOrderRequest, TrailingStopLossDetails)

from ta import trend
from ta import momentum

from oanda_order_creation import *

In [2]:
def convert_to_json(text):
    try:
        return ast.literal_eval(text)
    except:
#         print(text)
        return {}

In [3]:
def clean_candle_data(df, price_type = 'ask' ):
    
    df_modified = df[['complete', 'currency_pair', 'time', 'volume', price_type]]
    df_modified[price_type] = df_modified[price_type].astype(str)
    df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
    
    price_df = pd.json_normalize(df_modified[price_type])
    
    df_modified = pd.merge(df_modified, price_df,
                          left_index=True, right_index=True)
    
    df_modified = df_modified[['complete','currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
    df_modified.columns = ['complete','currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
    df_modified[["price_open", "price_high", "price_low", "price_close"]] = df_modified[["price_open", "price_high",
                                                                                         "price_low", "price_close"]].apply(pd.to_numeric)
    df_modified["time"] = pd.to_datetime(df_modified["time"])
    
    return df_modified

In [4]:
def get_macd_indicator(df, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False):
    # Innitialising MACD indicator
    indicator_macd = trend.MACD(close = df["price_close"])
    # Create columns for MACCD line, singal line and macd histogram (macd crosses signal line)
    df['macd'] = indicator_macd.macd()
    df['macd_signal'] = indicator_macd.macd_signal()
    df['macd_signal_diff'] = indicator_macd.macd_diff()
    return df

In [12]:
account_id = "101-003-18483320-004"
access_token = "87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5"

#### GET X NUMBER OF CANDLES FOR A SPECIFIC INTSTRUMENTS (REAL TIME FOR LAST CANDLE)

In [6]:
def get_candle_data(instrument, granularity, number_of_candles=200):
    client = oandapyV20.API(access_token=access_token, environment = "practice" or "live")
    params = {
      "count": number_of_candles,
      "granularity": granularity
    }
    r = instruments.InstrumentsCandles(instrument=instrument,
                                   params=params)
    response = client.request(r)
    df = pd.DataFrame(response["candles"])
    df["currency_pair"] = instrument
    return clean_candle_data(df, "mid")
    return df
    

In [11]:
extraction = False
while extraction == False:
#     time.sleep(0.5)
    time_now = datetime.datetime.utcnow()
    if (time_now.minute % 5 == 0) & (time_now.second == 0):
        print(time_now)
        extraction = True
        

time.sleep(5)
state = True
activity_dict = {}

while state == True:
    df = get_candle_data("EUR_USD", "M5", 500)

    df_with_indicators =  get_macd_indicator(df)

    indicator_sma = trend.SMAIndicator(close = df_with_indicators["price_close"], window=200)
    df_with_indicators['sma_200'] = indicator_sma.sma_indicator()

    indicator_rsi = momentum.RSIIndicator(close = df_with_indicators["price_close"], window=14)
    df_with_indicators['rsi_14'] = indicator_rsi.rsi()

    df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

    if ((df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and
        (df_with_indicators_complete["rsi_14"].iloc[-1] >= 30) and
        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00))):
        decision = "SELL"
        print(df_with_indicators_complete["time"].iloc[-1], "SELL")
        
        create_market_order_with_trailing_sl_only("EUR_USD", "SELL", 5000, 10)

    elif((df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and
        (df_with_indicators_complete["rsi_14"].iloc[-1] <= 70 ) and
        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00))):
        decision = "BUY"
        print(df_with_indicators_complete["time"].iloc[-1], "BUY")

        create_market_order_with_trailing_sl_only("EUR_USD", "BUY", 5000, 10)
    else:
        print(df_with_indicators_complete["time"].iloc[-1], "Nothing happens")
        decision = "NOTHING HAPPENS"
    activity_dict[df_with_indicators_complete["time"].iloc[-1]] = decision
    time.sleep(300)

2021-06-20 10:05:00
2021-06-18 20:55:00+00:00 Nothing happens
2021-06-18 20:55:00+00:00 Nothing happens


KeyboardInterrupt: 

In [ ]:
test < test2

In [10]:
test2 = datetime.time(21, 57)